In [1]:
!pip install zipfile
!pip install pandas
!pip install python-docx
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-instructor
!pip install chromadb
!pip install llama-index
!pip install transformers
!pip install torch

ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile


In [3]:
import zipfile
import pandas as pd
import os

data = zipfile.ZipFile('/content/Al Nassar Club_data corpus-20241106T131026Z-001.zip')
os.mkdir('/content/data')
data.extractall('/content/data')

In [5]:
import chromadb as chdb
from chromadb.utils import embedding_functions
chroma = chdb.PersistentClient(path="chroma_db_files")
collection_name = "Rag_using_HuggingFace_Embeddings"
collection = chroma.create_collection(name=collection_name)

In [6]:
def split_text_into_chunks(text, chunk_size=1000, chunk_overlap=20):
  chunks = []
  start = 0
  while start < len(text):
    end = start + chunk_size
    chunks.append(text[start:end])
    start = end
  return chunks

In [11]:
import os
from collections import defaultdict

def count_file_extensions(directory):
    extension_count = defaultdict(int)  # Dictionary to store counts of each file extension

    # Walk through all files in the directory and subdirectories
    for root, _, files in os.walk(directory):
        for file in files:
            # Split the file extension
            _, ext = os.path.splitext(file)
            ext = ext.lower()  # Convert to lowercase for uniformity
            extension_count[ext] += 1  # Increment count for this extension

    # Display results
    for ext, count in extension_count.items():
        print(f"{ext or 'No Extension'}: {count} files")

# Example usage
directory_path = '/content/data/Al Nassar Club_data corpus'  # Replace with your directory path
count_file_extensions(directory_path)


.txt: 12 files
.docx: 3 files
.json: 4 files
.csv: 4 files


In [15]:
import os
import json
import pandas as pd
from docx import Document

# Function to read and process text files
def read_txt(file_path):
    with open(file_path, 'r') as file:
        return file.read()

# Function to read and process docx files
def read_docx(file_path):
    doc = Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

# Function to read and process json files
def read_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
        return json.dumps(data)  # Convert JSON data to string for processing

# Function to read and process csv files
def read_csv(file_path):
    # Specify the encoding as 'latin-1' when reading the CSV file
    df = pd.read_csv(file_path, encoding='latin-1')
    return df.to_string()  # Convert DataFrame to string for processing

# Function to handle all files and split them into chunks
def process_directory(directory):
    combined_text = ""

    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)

        if filename.endswith('.txt'):
            combined_text += read_txt(file_path)
        elif filename.endswith('.docx'):
            combined_text += read_docx(file_path)
        elif filename.endswith('.json'):
            combined_text += read_json(file_path)
        elif filename.endswith('.csv'):
            combined_text += read_csv(file_path)

    # Split the combined text into chunks
    chunked_data = split_text_into_chunks(combined_text)
    print(f"Total chunks: {len(chunked_data)}")
    return chunked_data

# Process the directory
directory = '/content/data/Al Nassar Club_data corpus'
chunked_data = process_directory(directory)

Total chunks: 61


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.9/861.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.2.1
    Uninstalling sentence-transformers-3.2.1:
      Successfully uninstalled sentence-transformers-3.2.1


In [21]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

embeddings = embed_model.get_text_embedding_batch(chunked_data)
print(len(embeddings))
print(embeddings[:5])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

61
[[-0.04896034300327301, 0.018213016912341118, 0.018587032333016396, -0.005117232445627451, 0.031075743958353996, -0.02192954160273075, 0.008428027853369713, 0.018284959718585014, -0.016595054417848587, -0.04160642623901367, -0.028072170913219452, 0.02258886583149433, -0.04563728719949722, 0.010930404998362064, 0.04177413508296013, 0.05367576703429222, 0.007745819166302681, 0.05500786006450653, 0.05193956568837166, -0.01518450677394867, 0.010724902153015137, 0.03415348008275032, 0.051625143736600876, -0.033722005784511566, 0.05818308889865875, 0.03951139748096466, 0.01013562735170126, 0.008008763194084167, -0.05011465400457382, 0.007096568122506142, 0.026602376252412796, -0.05635576695203781, -0.024413634091615677, -0.014100709930062294, 0.036044005304574966, -0.03558892384171486, 0.06341330707073212, 0.03235426917672157, 0.017410797998309135, -0.025855647400021553, -0.07206686586141586, 0.022078802809119225, -0.06147834658622742, 0.01646946370601654, -0.02213028259575367, 0.01572561

In [23]:
ids = [f"chunk-{i}" for i in range(len(chunked_data))]
collection.upsert(
    documents=chunked_data,
    metadatas=None,
    embeddings=embeddings,
    ids=ids  # Use the generated IDs
)
from multiprocessing import context
def query_collection(query_text, n_results=2, top_k=3):
    query_embedding = embed_model.get_text_embedding(query_text)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )
    return results

In [24]:
query_collection(" Al Nassr was officially established on March 24, 1955, in Riyadh, Saudi Arabia." , n_results=1)

{'ids': [['chunk-5']],
 'embeddings': None,
 'documents': [[" of the prestigious tournament, showcasing their development and competitiveness.\nShort Narratives:\nClub Founded (1955): The vision of a few passionate individuals laid the foundation for what would become a leading football institution in Saudi Arabia. Al Nassr started as a local team but quickly rose to prominence, becoming a symbol of community pride and competitive spirit.\nFirst Saudi Pro League Title (1976): Winning the league title for the first time set the stage for future successes and established Al Nassr as a formidable contender in Saudi football. The victory inspired the players and fans, leading to increased support and growth of the club's following.\nManagement Change (1989): The introduction of new management brought innovative approaches, enhancing the team's performance and securing talented players. This strategic shift was crucial in adapting to the evolving landscape of football in the region.\nAsian 

In [25]:
# Assuming your collection is already created and you have the embeddings
for idx, embedding in enumerate(embeddings):
    collection.add(
        ids=[f"doc_{idx}"],  # Unique document ID
        embeddings=[embedding],  # The embedding vector
        metadatas=[{"source": "source_name"}],  # Optional metadata
        documents=[chunked_data[idx]]  # Corresponding document/text
    )

print("Embeddings saved to Chroma collection.")


Embeddings saved to Chroma collection.


In [26]:
import pandas as pd

# Assuming embeddings is a 2D list or array
embedding_filename = '/content/data/embeddings.csv'
df = pd.DataFrame(embeddings)

# Save as CSV
df.to_csv(embedding_filename, index=False)

print(f"Embeddings saved to {embedding_filename}")


Embeddings saved to /content/data/embeddings.csv


In [31]:
import shutil

# Specify the directory to zip
directory_to_zip = '/content/chroma_db_files'
# Specify the path for the resulting zip file (without the extension)
zip_filename = '/content/Chroma'

# Create a zip archive of the directory
shutil.make_archive(zip_filename, 'zip', directory_to_zip)

print(f"Directory zipped and saved as {zip_filename}.zip")


Directory zipped and saved as /content/Chroma.zip


In [40]:
def generate_answer(query, retrieved_docs):
    tokenizer.pad_token = tokenizer.eos_token

    flattened_docs = [doc for sublist in retrieved_docs for doc in sublist]
    prompt = query + "\n" + " ".join(flattened_docs)

    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True, padding=True)

    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=150,
        num_return_sequences=1,
        no_repeat_ngram_size=2
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

query = "Write me a poem about Al Nassr, the pride of Saudi football."
retrieved_docs = query_collection(query)
answer = generate_answer(query, retrieved_docs)

print(answer)


Write me a poem about Al Nassr, the pride of Saudi football.
, we sing loud,
Together, we are so proud!

5. "Champions of the League"
Description:
Sung to celebrate the team’s achievements, especially during title-winning seasons.

Lyrics:
Champions of the League,
We’ll raise the trophy high,
With our heads held up high,
Al Nassr, you’ll never die!Legends
1. Majed Abdullah
Position: Forward
Playing Career: 1976–1998
Achievements:
All-time top scorer for Al Nassr with over 200 goals.
Winner of the AFC Asian Cup with the Saudi national team in 1988.
Named Saudi Arabia’s Player of the Year multiple times.
Legacy:
Regarded as one of the greatest Arab players in history, Abdullah’s skill and scoring ability helped elevate Al Nassr’s status in domestic and international competitions.
Inducted into the Saudi Arabian Football Hall of Fame.
2. Sami Al-Jaber
Position: Forward
Playing Career: 1989–2007
Achievements:
Scored over 180 goals for Al Nassr.
Participated in four World Cups (1994, 1998, 